<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

address = "University/Term 6/MIR/Homeworks/HW3"
import sys
sys.path.append(f"/content/drive/My Drive/{address}")

%cd /content/drive/My\ Drive/$address

Mounted at /content/drive
/content/drive/My Drive/University/Term 6/MIR/Homeworks/HW3


In [ ]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
!pip install hazm
!pip install unidecode
# !pip install pandas
# !pip install nltk
# !pip install torch
# !pip install transformers
!pip install sentence_transformers
!pip install faiss-cpu
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 32.7 MB/s 
     |████████████████████████████████| 233 kB 69.1 MB/s 
     |████████████████████████████████| 1.4 MB 55.8 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394491 sha256=a1d311b7757368ec47c90f73a6cc40b6699f38e41f8b23cd76f291c68fc5b54d
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154510 sha256=a1eda14d7f7cabdd5f86441ac96a430f37c3edcbe214d74256503057e35e95d6
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
Looking in indexes: h

In [ ]:
import os
import re
import csv
import hazm
import nltk
import zipfile
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display
from transformers import pipeline

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٨٠ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [ ]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [ ]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            category_PER = soup.find_all('li', {'class': 'breadcrumb-item'})
            category_PER = list(map(lambda x: x.text.strip(), category_PER))[1:]
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category-PER': category_PER,
                'category-ENG': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [ ]:
scraper = Scraper(current_year=1401, current_month=3)
# scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [ ]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    return dataset


dataset = read_dataset_from_file()

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [ ]:
def display_dataset_info():
    global CATEGORIES

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for data in dataset:
        length_dict[data['category-ENG']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,80986
1,سیاسی,15810
2,جهانی,4800
3,اقتصادی,9873
4,اجتماعی,14070
5,شهری,4260
6,ورزشی,8421
7,علمی,3267
8,فرهنگی,6657
9,فناوری اطلاعات,444


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. در ادامه، به توضیح هر یک از مراحل پیش‌پردازش می‌پردازیم.
    </p>
</div>

In [ ]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

In [ ]:
import json

In [ ]:
def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()

def get_preprocessed_texts(texts, mode):
    if mode == 'save':
        preprocessor = Preprocessor(stopwords_path='stopwords.txt')
        preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
        with open('preprocessed_texts.json', 'w', encoding='utf-8') as file:
            json.dump(preprocessed_texts, file)
            return preprocessed_texts
    if mode == 'load':
        with open('preprocessed_texts.json', encoding='utf-8') as file:
            preprocessed_texts = json.load(file)
            return preprocessed_texts

texts = [data_to_text(data) for data in dataset]
preprocessed_texts = get_preprocessed_texts(texts, mode='load')

In [ ]:
# pd.DataFrame(preprocessed_texts).to_csv(f'preprocessed_texts.csv', encoding='utf-8')

In [ ]:
dataset = pd.DataFrame(dataset)

In [ ]:
dataset

,date,title,intro,body,category-PER,category-ENG
0,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۴۰,واشنگتن: آمریکا و ایران هدف مشترکی دارند,سخنگوی وزارت خارجه آمریکا شامگاه سه‌شنبه در کن...,به گزارش همشهری‌آنلاین به نقل از فارس، ند پرای...,"['سياست', 'سیاست\u200cخارجی']",Politics
1,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۳۱,عراقچی باز هم ادعاهای «منبع مطلع» پرس‌تی‌وی در...,معاون وزیر امور خارجه جمهوری اسلامی ایران در ت...,به گزارش همشهری‌آنلاین به نقل از ایرنا، عباس ع...,"['سياست', 'سیاست\u200cخارجی']",Politics
2,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۲۰,هشدار به دیپلمات‌های آمریکا در چاد؛ مراقب جان ...,وزارت خارجه آمریکا با صدور بیانیه‌ای به تمام د...,به گزارش همشهری آنلاین به نقل از فارس، به دنبا...,"['جهان', 'آمریکا']",Politics
3,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۷,نامه مشاور رهبری به نمکی درباره واکسن پولی,از درگیر کردن بخش خصوصی به شدت پرهیز شود، چه د...,به گزارش همشهری آنلاین به نقل از ایسنا، در نام...,"['سياست', 'سیاست داخلی']",Politics
4,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۵,درخواست از روحانی: کشور را قرنطینه کامل کنید,حزب اتحاد ملت در نامه‌ای خطاب به روحانی خواستا...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، مت...,"['سياست', 'سیاست داخلی']",Politics
...,...,...,...,...,...,...
80981,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۵۴,از زینک چه می‌دانید؟ | علائم کمبود زینک یا روی,بدن شما نیاز به انواع مواد مغذی دارد تا عملکرد...,به گزارش همشهری آنلاین، بدن شما نیاز به انواع ...,"['زندگی', 'تندرستی']",LifeSkills
80982,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۰۲,یک راهکار ساده برای مطمئن شدن از کیفیت عینک آف...,استفاده از عینک آفتابی یک کار ضروری برای محافظ...,به گزارش همشهری آنلاین، بررسی‌ها نشان داده قرا...,"['زندگی', 'مهارت\u200cهای زندگی']",LifeSkills
80983,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۸:۰۰,تصاویری از سرزمین لک‌لک‌های ایران | زریوار میز...,بهار فصل خوبی برای سفر به کردستان است؛ جاده‌ها...,به گزارش همشهری آنلاین، شنیدن صدای لک‌لک‌ها یک...,"['زندگی', 'پیشنهاد سفر']",LifeSkills
80984,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۷:۵۵,۱۰ ترفند خانگی که سردردتان را بدون نیاز به قرص...,سردرد مشکل نادری نیست و همه‌ ما گاهی دچارش می‌...,به گزارش همشهری آنلاین به نقل از سلامت نیوز، ت...,"['زندگی', 'تندرستی']",LifeSkills


In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

# v3.0
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# model = TFAutoModel.from_pretrained(model_name_or_path)  For TF
model = AutoModel.from_pretrained(model_name_or_path)

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.poo

In [ ]:

text = "ما در هوش‌واره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد میتوانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است."
tokenizer.tokenize(text)

['ما', 'در', 'هوش', '[ZWNJ]', 'واره', 'معتقدیم', 'با', 'انتقال', 'صحیح', 'دانش', 'و', 'آ', '##گاهی', '،', 'همه', 'افراد', 'میتوانند', 'از', 'ابزارهای', 'هوشمند', 'استفاده', 'کنند', '.', 'شعار', 'ما', 'هوش', 'مصنوعی', 'برای', 'همه', 'است', '.']

['ما',
 'در',
 'هوش',
 '[ZWNJ]',
 'واره',
 'معتقدیم',
 'با',
 'انتقال',
 'صحیح',
 'دانش',
 'و',
 'آ',
 '##گاهی',
 '،',
 'همه',
 'افراد',
 'میتوانند',
 'از',
 'ابزارهای',
 'هوشمند',
 'استفاده',
 'کنند',
 '.',
 'شعار',
 'ما',
 'هوش',
 'مصنوعی',
 'برای',
 'همه',
 'است',
 '.']

In [ ]:

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

In [ ]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('HooshvareLab/bert-fa-zwnj-base')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab_bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab

cuda:0


In [ ]:
# Convert abstracts to vectors
embeddings = model.encode(dataset.title.to_list(), show_progress_bar=True)

Batches:   0%|          | 0/2531 [00:00<?, ?it/s]

In [ ]:
print(f'Shape of the vectorised abstract: {embeddings[0].shape}')

Shape of the vectorised abstract: (768,)


In [ ]:
old_embeddings = embeddings

# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, np.array(list(range(len(dataset)))))

print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 80986


In [ ]:
# pd.DataFrame(embeddings).to_csv("title_embeddings", encoding='utf-8')

In [ ]:
dataset.iloc[64017, 1]

'خداحافظ المپیک بهترین فیلم جشنواره ایرلندی شد'

In [ ]:
# Retrieve the 10 nearest neighbours
D, I = index.search(np.array([embeddings[64017]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\nNews IDs: {I.flatten().tolist()}')

L2 distance: [0.0, 113.06363677978516, 133.37811279296875, 146.45591735839844, 151.03857421875, 159.95999145507812, 168.0147247314453, 168.1057891845703, 174.82716369628906, 177.13995361328125]
News IDs: [64017, 61777, 66383, 64912, 61296, 64894, 62374, 66550, 61738, 64318]


In [ ]:
def vector_search(query, model, index, num_results=10):
    """Tranforms query to vector using a pretrained, sentence-level 
    DistilBERT model and finds similar vectors using FAISS.
    Args:
        query (str): User query that should be more than a sentence long.
        model (sentence_transformers.SentenceTransformer.SentenceTransformer)
        index (`numpy.ndarray`): FAISS index that needs to be deserialized.
        num_results (int): Number of results to return.
    Returns:
        D (:obj:`numpy.array` of `float`): Distance between results and query.
        I (:obj:`numpy.array` of `int`): Paper ID of the results.
    
    """
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I

In [ ]:
def i_to_title(i, dataset: pd.DataFrame):
    return dataset.iloc[i].title

In [ ]:
user_query = "گلکسی سامسونگ"

In [ ]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\nNews IDs: {I.flatten().tolist()}')

L2 distance: [335.1736755371094, 335.1737976074219, 360.69378662109375, 364.5807800292969, 375.4864501953125, 383.4737548828125, 384.74810791015625, 394.5172119140625, 402.2670593261719, 402.5286865234375]
News IDs: [78821, 59377, 26520, 67169, 58007, 27719, 57372, 67393, 30251, 27706]


In [ ]:
found_titles = [i_to_title(i, dataset) for i in I.flatten().tolist()]

In [ ]:
found_titles

['بررسی و مقایسه گوشی\u200cهای سامسونگ و شیائومی',
 'بررسی و مقایسه گوشی\u200cهای سامسونگ و شیائومی',
 'جدیدترین قیمت انواع گوشی موبایل در بازار | آخرین قیمت آیفون ۱۳ و سامسونگ سری گلکسی',
 'سامسونگ تراشه اگزینوس ۲۲۰۰ خود را معرفی کرد',
 'قیمت احتمالی گلکسی A12s سامسونگ اعلام شد',
 'جدیدترین قیمت انواع موبایل پنجشنبه ۲ دی ۱۴۰۰ | سامسونگ و شیائومی چند؟ |  جدول قیمت\u200cها',
 'آشنایی با رمزهای مخفی تلفن\u200cهای سامسونگ',
 'آزمایش سقوط برای سری گالکسی اس ۲۲ | آیفون بادوام\u200cتر است یا سامسونگ؟',
 'جدیدترین قیمت گوشی موبایل در اول اردیبهشت ۱۴۰۱ | از سامسونگ و شیائومی تا اپل و هوآوی',
 'جدیدترین قیمت موبایل\u200cهای سامسونگ و ال\u200cجی در بازار | جدول قیمت\u200cها']

In [ ]:
class Transformer:

    def __init__(self):
        self.model = SentenceTransformer('HooshvareLab/bert-fa-zwnj-base')
        if torch.cuda.is_available():
            self.model = self.model.to(torch.device("cuda"))
        self.embeddings = None
        self.index = None
    
    def train_embeddings(self, dataset):
        self.embeddings = self.model.encode(dataset, show_progress_bar=True)
        self.embeddings = np.array(
            [embedding for embedding in self.embeddings]).astype("float32")
    
    def make_index(self, dataset):
        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index = faiss.IndexIDMap(self.index)
        self.index.add_with_ids(self.embeddings, np.array(list(range(len(dataset)))))

    def save_embeddings(self, path):
        pd.DataFrame(self.embeddings).to_csv(path, encoding='utf-8')
    
    def predict(self, query, dataset, k=10):
        vector = self.model.encode(list(query))
        D, I = self.index.search(np.array(vector).astype("float32"), k=k)
        return [dataset.iloc[i] for i in I.flatten().tolist()]
        

In [ ]:
transformer = Transformer()

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab_bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab

In [ ]:
joined_preprocessed_texts = []
for i in range(len(preprocessed_texts)):
    joined_preprocessed_texts.append(' '.join(preprocessed_texts[i]))

In [ ]:
joined_preprocessed_texts[1]

'عراقچی ادعاهای منبع مطلع پرس\u200cتی\u200cوی مذاکرات وین تکذیب معاون وزیر امور خارجه جمهوری اسلامی ایران توئیتی ادعاهای مطرح\u200cشده شبکه تلویزیونی پرس\u200cتی\u200cوی نقل منبع مطلع تکذیب اعلام نمی\u200cدانم منبع مطلع پرس\u200cتی\u200cوی وین کیست مطلع گزارش نقل ایرنا عباس عراقچی معاون وزیر خارجه رئیس تیم مذاکره\u200cکننده ایران وین پیامی توئیتری نوشت نمی\u200cدانم منبع مطلع پرس\u200cتی\u200cوی وین کیست مطلع امروز پرس تی نقل منبعی مطلع ادعاها مذاکرات وین مطرح وزارت خارجه بارها تاکید اخبار مذاکرات وزارت خارجه رسمی دنبال'

In [ ]:
transformer.train_embeddings(dataset.title.to_list())

Batches:   0%|          | 0/2531 [00:00<?, ?it/s]

In [ ]:
transformer.make_index(dataset.title.to_list())

In [ ]:
transformer.save_embeddings("new_title_embeddings.csv")

In [ ]:
pd.DataFrame(transformer.predict(["فدراسیون فوتبال"], dataset))

,date,title,intro,body,category-PER,category-ENG
31897,جمعه ۱۷ اردیبهشت ۱۴۰۰ - ۱۰:۴۴,تشکیل تیم فوتبال بهبودیافتگان,اداره ورزش شهرداری منطقه ۱۶ قصد دارد از میان ب...,مدیر اداره ورزش شهرداری منطقه ۱۶ با بیان اینکه...,"['محله', 'زیر پوست شهر']",Society
79256,پنجشنبه ۲۳ دی ۱۴۰۰ - ۰۱:۳۲,عکس روز| فوتبال قطع عضو,بازیکنان فوتبال قطع عضو در یک مسابقه مقدماتی د...,فوتبال قطع عضو گونه‌ای از فوتبال است که برای ا...,"['زندگی', 'چهره\u200cها']",LifeSkills
55247,پنجشنبه ۷ بهمن ۱۴۰۰ - ۱۷:۵۳,ویدیو | صحبت های رئیس فدراسیون فوتبال در ورزشگ...,رئیس فدراسیون فوتبال در بدو ورود به ورزشگاه آز...,به گزارش همشهری آنلاین، شهاب الدین عزیزی خادم ...,"['ورزش', 'فوتبال ايران']",Sport
74488,پنجشنبه ۱۸ شهریور ۱۴۰۰ - ۱۶:۲۶,قول رئیس فیفا به هواداران فوتبال ایران و عزیزی...,رئیس فدراسیون جهانی فوتبال برای هواداران در ای...,به گزارش همشهری‌آنلاین، جیوانی اینفانتینو، رئی...,"['ورزش', 'فوتبال ايران']",LifeSkills
52070,پنجشنبه ۱۸ شهریور ۱۴۰۰ - ۱۶:۲۶,قول رئیس فیفا به هواداران فوتبال ایران و عزیزی...,رئیس فدراسیون جهانی فوتبال برای هواداران در ای...,به گزارش همشهری‌آنلاین، جیوانی اینفانتینو، رئی...,"['ورزش', 'فوتبال ايران']",Sport
52815,سه‌شنبه ۶ مهر ۱۴۰۰ - ۱۰:۳۲,محکومیت باشگاه استقلال در کمیته تعیین وضعیت فد...,باشگاه استقلال در دو پرونده از سوی کمیته تعیین...,به گزارش همشهری‌آنلاین به نقل از سایت فدراسیون...,"['ورزش', 'فوتبال ايران']",Sport
50769,شنبه ۱۹ تیر ۱۴۰۰ - ۱۲:۲۴,عکس | تجمع هواداران ذوب‌آهن مقابل فدراسیون فوتبال,تعدادی از هواداران تیم فوتبال ذوب‌آهن اصفهان م...,به گزارش همشهری‌آنلاین به نقل از فارس، جمعی از...,"['ورزش', 'فوتبال ايران']",Sport
53521,چهارشنبه ۲۴ آذر ۱۴۰۰ - ۱۵:۲۷,آشنایی با سازمان لیگ فوتبال ایران,سازمان لیگ فوتبال ایران در ۱۱ آبان ۱۳۸۰ شکل گر...,به گزارش همشهری آنلاین، سازمان لیگ فوتبال ایر...,"['ورزش', 'فوتبال ايران']",Sport
54743,پنجشنبه ۲۸ بهمن ۱۴۰۰ - ۱۷:۳۶,سرپرست فدراسیون فوتبال نیامده تغییر کرد,در حالی که صبح امروز سرپرست فدراسیون فوتبال تع...,به گزارش همشهری آنلاین پس از عزل موقت شهاب‌الد...,"['ورزش', 'فوتبال ايران']",Sport
49619,شنبه ۱۸ اردیبهشت ۱۴۰۰ - ۱۲:۰۴,عکس | نماز جماعت رقیب استقلال در زمین فوتبال,اقدام عبادی مربی و بازیکنان الهلال پیش از دیدا...,به گزارش همشهری‌آنلاین، تیم فوتبال الهلال عربس...,"['ورزش', 'فوتبال ايران']",Sport


In [ ]:
title_transformer = transformer

In [ ]:
transformer = Transformer()

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab_bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab

In [ ]:
transformer.train_embeddings(joined_preprocessed_texts)

Batches:   0%|          | 0/2531 [00:00<?, ?it/s]

In [ ]:
transformer.save_embeddings("processed_embeddings.csv")